<a href="https://colab.research.google.com/github/BatoolMM/fastai/blob/master/chapter__7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
!pip install fastcore==1.0.9
!pip install utils
!pip install wandb
!pip install nbdev

In [3]:
from utils import *
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner
import wandb
from fastai.callback.wandb import WandbCallback
import matplotlib.pyplot as plt

In [4]:
path = untar_data(URLs.IMAGENETTE)

In [5]:
path = path/'val'
path.ls()

(#10) [Path('/root/.fastai/data/imagenette2/val/n01440764'),Path('/root/.fastai/data/imagenette2/val/n03394916'),Path('/root/.fastai/data/imagenette2/val/n03000684'),Path('/root/.fastai/data/imagenette2/val/n03425413'),Path('/root/.fastai/data/imagenette2/val/n02979186'),Path('/root/.fastai/data/imagenette2/val/n03888257'),Path('/root/.fastai/data/imagenette2/val/n03445777'),Path('/root/.fastai/data/imagenette2/val/n02102040'),Path('/root/.fastai/data/imagenette2/val/n03417042'),Path('/root/.fastai/data/imagenette2/val/n03028079')]

In [9]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items = get_image_files,
                   get_y = parent_label,
                   item_tfms = Resize(460),
                   batch_tfms = aug_transforms(size=224, min_scale=0.75))


In [10]:
dls = dblock.dataloaders(path, bs=64)

In [12]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.489181,4.035932,0.268790,00:41
1,1.861379,1.742584,0.454777,00:41
2,1.514685,1.957198,0.480255,00:41
3,1.257581,1.040606,0.650955,00:42
4,1.077544,0.967823,0.668790,00:41
